- Sacar datos luz REAPI por mes (2019-2022): https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=month&geo_limit=ccaa
    - El precio en españa es el mismo para todos
- Sacar datos gas: https://data.cnmc.es/energia/mercado-mayorista-del-gas-natural/conjuntos-de-datos/evolucion-de-los-precios-por-productos

In [ ]:
# Importamos las librerías que necesitamos

# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Manipulación de datos en estructuras DataFrame para análisis y transformación
import numpy as np              # Realización de cálculos numéricos y manipulación de arrays multidimensionales
import datetime                 # Manejo y procesamiento de fechas y horas, útil para organizar datos temporalmente

# Manejo del tiempo y generación de pausas
# -----------------------------------------------------------------------
from time import sleep          # Pausa la ejecución del código durante un tiempo determinado, útil para evitar bloqueos en scraping
import random                   # Generación de valores aleatorios, ideal para espaciar solicitudes en procesos de scraping

# Para importar módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")          # Añade el directorio padre al PATH para permitir la importación de módulos personalizados
from src.SupportApi import (    # Importa funciones personalizadas de soporte
    api_gas,                    # Función para hacer llamadas a la API de gas
    api_luz                     # Función para hacer llamadas a la API de electricidad
)

# Control de advertencias
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')  # Suprime advertencias para mantener la salida de consola más limpia

# Para realizar solicitudes HTTP
import requests                  # Realiza solicitudes HTTP a APIs y obtiene datos en formato JSON


# Obtener Precios gas

Empezamos por obtener todos los precios del gas de 2019 a 2022

la función de abajo nos permite obtener los datos de 2019 a 2022. o en un rango de 2015 a 2023 que son los datos que se tienen hasta la fecha

In [21]:
df_gas = api_gas("2018-12-30","2022-12-01")
df_gas

,_id,fecha_de_negociacion,producto_de_mibgas,precios_de_los_productos_mibgas
2601,2602,2018-12-31,Producto intradiario WD,25.14
2602,2603,2018-12-31,Producto diario D+1,24.28
2603,2604,2018-12-31,Producto mensual M+1,24.10
2604,2605,2019-01-01,Producto intradiario WD,25.09
2605,2606,2019-01-01,Producto diario D+1,27.17
...,...,...,...,...
7228,7229,2022-11-29,Producto trimestral Q+1,104.00
7229,7230,2022-11-30,Producto intradiario WD,125.35
7230,7231,2022-11-30,Producto diario D+1,118.27
7231,7232,2022-11-30,Producto mensual M+1,122.37


Ahora vamos a dejar solo el producto de mibgas que nos interesa actualmente los productos que hay son:

In [22]:
df_gas["producto_de_mibgas"].unique()

array(['Producto intradiario WD', 'Producto diario D+1',
       'Producto mensual M+1', 'Producto mensual M+2',
       'Producto anual Y+1', 'Producto trimestral Q+1',
       'Producto trimestral Q+3', 'Producto mensual  M+3',
       'Producto trimestral Q+2'], dtype=object)

Vamos entonces a filtrar por el producto mensual M + 1, que nos va a proveer del precio del gas en el mes siguiente (Month + 1)

In [23]:
cond = df_gas["producto_de_mibgas"] == "Producto mensual M+1"
df_gas = df_gas[cond]

Lo guardamos, con objetivo de guardar nuestro progreso actual

In [24]:
df_gas.to_csv("../datos/01_PreciosGas/01_PrecioGasMensualSinLimpiar2019-2022.csv")

## Limpieza Datos Gas
Dado que hay múltiples precios en el mismo mes, nos vamos a quedar con el último precio registrado de cada mes

Primero ordenamos por fecha para asegurarnos de que están ordenados

In [25]:
df_gas = df_gas.sort_values(by="fecha_de_negociacion")

Ahora, creamos una nueva columna que nos deja solo el mes y el año

In [26]:
df_gas["anio_mes"] = df_gas["fecha_de_negociacion"].dt.to_period("M")
df_gas.sample()

,_id,fecha_de_negociacion,producto_de_mibgas,precios_de_los_productos_mibgas,anio_mes
3759,3760,2019-11-29,Producto mensual M+1,15.16,2019-11


Agrupamos por el último precio de cada mes y reseteamos el index, también generamos un index compatible para 

In [27]:
df_gas = df_gas.groupby("anio_mes").tail(1).reset_index(drop=True)
df_gas.index = df_gas.index + 1
df_gas.reset_index(inplace=True)
df_gas.sample()

,index,_id,fecha_de_negociacion,producto_de_mibgas,precios_de_los_productos_mibgas,anio_mes
34,35,6062,2021-10-29,Producto mensual M+1,71.14,2021-10


Guardamos nuestro progreso y continuamos

In [28]:
df_gas.to_csv("../datos/01_PreciosGas/02_PrecioGasPorMesSinLimpiar.csv")

Ahora que ya tenemos los precios, por simplicidad, vamos a marcar correctamente a que mes y año pertenecen para poder limpiar más los datos, hacemos un offset de un mes

In [29]:
df_gas["fecha_de_negociacion"] = df_gas["fecha_de_negociacion"] + pd.DateOffset(months=1)

### Eliminamos las columnas innecesarias y generamos correctamente la columna aniomes

In [30]:
df_gas.drop(columns=["_id","producto_de_mibgas","anio_mes"],inplace=True)
df_gas.head()

,index,fecha_de_negociacion,precios_de_los_productos_mibgas
0,1,2019-01-31,24.10
1,2,2019-02-28,22.73
2,3,2019-03-28,19.29
3,4,2019-04-29,15.12
4,5,2019-05-30,15.95


In [31]:
df_gas["fecha_de_negociacion"] = pd.to_datetime(df_gas["fecha_de_negociacion"]).dt.to_period("M").dt.to_timestamp()

In [32]:
df_gas = df_gas.rename(columns={"fecha_de_negociacion": "fecha",
                                "precios_de_los_productos_mibgas":"precio_gas"})
df_gas = df_gas.assign(fecha=df_gas.pop("fecha"))
df_gas.head()

,index,precio_gas,fecha
0,1,24.10,2019-01-01
1,2,22.73,2019-02-01
2,3,19.29,2019-03-01
3,4,15.12,2019-04-01
4,5,15.95,2019-05-01


In [33]:
df_gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   index       48 non-null     int64         
 1   precio_gas  48 non-null     float64       
 2   fecha       48 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.3 KB


Guardamos como producto Final y pasamos a la siguiente API

In [34]:
df_gas.to_csv("../datos/01_PreciosGas/03_TablaPreciosGasFinal.csv")

# REData API (Red Eléctrica de España)

Vamos a obtener el precio mensual de la electricidad en España de 2019 a 2022 para ver la evolución
para ello haremos peticiones a esta url:
- https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=month&geo_limit=ccaa

Donde vamos a modificar start_date y end_date en varias ocasiones para poder obtenerlo, ya que por limitación, la API solo nos da de año en año
Con la siguiente función unimos cada request al DataFrame, donde solo nos interesa el precio y el año y mes

In [35]:
years = [2019,2020,2021,2022]
df_luz = api_luz(years,"../datos/02_PreciosLuz")

Realizando Request a: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2019-01-01T00:00&end_date=2019-12-31T23:59&time_trunc=month&geo_limit=ccaa
Backup Realizado: PreciosLuz2019.json
Realizando Request a: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2020-01-01T00:00&end_date=2020-12-31T23:59&time_trunc=month&geo_limit=ccaa
Backup Realizado: PreciosLuz2020.json
Realizando Request a: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2021-01-01T00:00&end_date=2021-12-31T23:59&time_trunc=month&geo_limit=ccaa
Backup Realizado: PreciosLuz2021.json
Realizando Request a: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=month&geo_limit=ccaa
Backup Realizado: PreciosLuz2022.json
Dataframe guardado en: ../datos/02_PreciosLuz


In [36]:
df_luz.head()

,index,precio,fecha
0,1,67.91,2019-01-01T00:00:00.000+01:00
1,2,59.79,2019-02-01T00:00:00.000+01:00
2,3,54.08,2019-03-01T00:00:00.000+01:00
3,4,56.54,2019-04-01T00:00:00.000+02:00
4,5,53.71,2019-05-01T00:00:00.000+02:00


In [37]:
df_luz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   48 non-null     int64  
 1   precio  48 non-null     float64
 2   fecha   48 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.3+ KB


### Pasar a formato Datetime

In [ ]:
df_luz["fecha"] = df_luz["fecha"].str.split("T").str[0]
df_luz["fecha"] = pd.to_datetime(df_luz["fecha"],format="%Y-%m-%d",utc=False)
df_luz.head()

,index,precio,fecha
0,1,67.91,2019-01-01
1,2,59.79,2019-02-01
2,3,54.08,2019-03-01
3,4,56.54,2019-04-01
4,5,53.71,2019-05-01


In [39]:
df_luz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   48 non-null     int64         
 1   precio  48 non-null     float64       
 2   fecha   48 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.3 KB


### Lo Guardamos y continuamos

In [40]:
df_luz.to_csv("../datos/02_PreciosLuz/02_PreciosLuzFinal.csv")